# Import Libraries

In [4]:
from sklearn.cluster import KMeans
from config import *
import pandas as pd
import numpy as np
import random
from collections import defaultdict
import matplotlib.pyplot as plt

NUM_CLUSTERS = 5

# Generate Locations (Demand Point, Charging Points, Depot)

In [7]:
random.seed(SEED)

# Electric Vehicle Points
ev_pts = pd.read_csv("Unique_EV_Points.csv")
charge_pts = ev_pts.sample(n=NUM_CHARGERS)[['Longitude', 'Latitude']] # dataframe
charge_pts_list = list(charge_pts.itertuples(index=False, name=None)) # list

# Delivery locations Points
dl = pd.read_csv("DeliveryLocations.csv")
todays_locations = dl.sample(n=NUM_CUSTOMERS)
dl_list = list(todays_locations.itertuples(index=False, name=None))

# Fedex Ship Centre
fedex_centre = [(104.0023106, 1.3731437)]

locations = fedex_centre + dl_list + charge_pts_list # combined list of all locations
print(locations)

[(104.0023106, 1.3731437), (103.8051481, 1.441131263), (103.8435933, 1.371223221), (103.7748564, 1.374941101), (103.7751648, 1.443151211), (103.7472473, 1.320083142), (103.8571215, 1.305734406), (103.7559635, 1.349336423), (103.8427432, 1.36166764), (103.8363183, 1.421334173), (103.7976719, 1.438012808), (103.8376878, 1.370096538), (103.8892534, 1.391426582), (103.808062, 1.318832253), (103.9242187, 1.333580162), (103.8072294, 1.273494356), (103.9337674, 1.347313579), (103.9567911, 1.353463173), (103.9543247, 1.366184324), (103.8786414, 1.378183076), (103.7392387, 1.322857732), (103.7958352, 1.430140406), (103.744795, 1.351252319), (103.9117112, 1.333122056), (103.7374828, 1.321009823), (103.7820031, 1.430459509), (103.8295591, 1.281962435), (103.900666, 1.403145259), (103.7050502, 1.33607704), (103.8516986, 1.361342556), (103.9105619, 1.396185406), (103.945859, 1.351414561), (103.8192532, 1.44625183), (103.7106998, 1.345057736), (103.762395, 1.306287483), (103.8844479, 1.351624426), (

# K-means clustering

In [10]:
X = np.array(dl_list) # convert to numpy array, prepare for clustering
kmeans = KMeans(n_clusters = NUM_CLUSTERS, random_state = SEED)
labels = kmeans.fit_predict(X)

clustered_customers = defaultdict(list)
for idx, label in enumerate(labels):
    clustered_customers[label].append(dl_list[idx])

# Visualise clusters, charging point and depot

In [13]:
import folium
from folium.plugins import MarkerCluster
from matplotlib import cm

# Center the map on Singapore (or your depot)
m = folium.Map(location=fedex_centre[0][::-1], zoom_start=12)  # Note: folium uses (lat, lon)

# Color map for clusters (tab10 is fine for <=10 clusters)
colormap = cm.get_cmap('tab10', NUM_VEHICLES)

# Add clustered demand points
for cluster_id, points in clustered_customers.items():
    color = "#{:02x}{:02x}{:02x}".format(
        int(colormap(cluster_id)[0]*255),
        int(colormap(cluster_id)[1]*255),
        int(colormap(cluster_id)[2]*255)
    )
    for lat, lon in points:
        folium.CircleMarker(
            location=(lon, lat),
            radius=5,
            color=color,
            fill=True,
            fill_opacity=0.8
        ).add_to(m)

# Add depot
folium.Marker(
    location=fedex_centre[0][::-1],
    popup='Depot',
    icon=folium.Icon(color='red', icon='home')
).add_to(m)

# Add charger points
for lat, lon in charge_pts_list:
    folium.Marker(
        location=(lon, lat),
        popup='Charger',
        icon=folium.Icon(color='green', icon='flash')
    ).add_to(m)

# Display map
m

# Assigning vehicles to each cluster

Assign vehicles to clusters proportionally to the number of vehicles in each cluster.

- Count customer in each cluster

- Compute share of each vehicle

- Allocate vehicles using share

- Ensure solution is feasible

<class 'collections.defaultdict'>
